Transform summary statistics file from trafficstop_county_summarystat.ipynb to format suitable for tableau 

Output file format:

1. **tableau_cite_rate_spd_cat.xlsx**:

We deleted rows that have at least one null values (rate = null indicates that too few (0) records for a speeding type for a county)

columns:

race | citation_rate	| county | type

2. **'tableau_other_pct.xlsx'**:

columns:

race | citation_rate	| county | type

(Note: here citation_rate is actually 'rate'. Definition is based on type - defined inn trafficstop_county_summarystat.ipynb)

3. **'speeding_latlng.xlsx'**

speeding_stops | county | Latitude | Longitude | Population(2010)

with county_info (latitude, longitude, population (2010)) from **table_county.csv**.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import drive
import os 
drive.mount('/gdrive/')

Mounted at /gdrive/


#### For first citation rate file

In [ ]:
file_name = 'cite_rate_spd_cat.csv'
path = "/gdrive/MyDrive/traffic_stop/TX-county/summarystat/"
cite_df = pd.read_csv((path+file_name))
cite_df

,asian/pacific islander,black,hispanic,white,county,type
0,0.000000,0.000000,0.073529,0.017167,Hays County,cite_rate_unsafe speed
1,0.000000,0.000000,0.008475,0.010563,Wilson County,cite_rate_unsafe speed
2,NaN,0.166667,0.333333,0.058824,Tarrant County,cite_rate_unsafe speed
3,0.000000,0.011236,0.000000,0.023750,Gregg County,cite_rate_unsafe speed
4,0.000000,0.000000,0.052632,0.051724,Jackson County,cite_rate_unsafe speed
...,...,...,...,...,...,...
1011,0.000000,0.014493,0.000926,0.000862,Hansford County,cite_rate_speeding over limit
1012,0.028571,0.007042,0.020718,0.012461,Lynn County,cite_rate_speeding over limit
1013,NaN,0.000000,0.036810,0.008818,Cochran County,cite_rate_speeding over limit
1014,NaN,0.000000,0.054054,0.023256,Loving County,cite_rate_speeding over limit


In [ ]:
# Select rows which do not contain  any NaN value in any of the column
filtered_df = cite_df[~cite_df.isnull().any(axis=1)]

In [ ]:
file_name = 'filt_cite_rate_spd_cat.xlsx'
path = "/gdrive/MyDrive/traffic_stop/TX-county/summarystat/"
save_path = path + file_name
filtered_df.to_excel(save_path, index = False)

In [ ]:
filtered_df['type'].value_counts()

cite_rate_speeding over limit                        252
cite_rate_speeding-10% or more above posted speed    251
cite_rate_unsafe speed                               174
cite_rate_fail to control speed                      136
Name: type, dtype: int64

In [ ]:
list(filtered_df.columns)

['asian/pacific islander', 'black', 'hispanic', 'white', 'county', 'type']

In [ ]:
new_filt_df = filtered_df
df1 = new_filt_df.loc[new_filt_df['type'] == 'cite_rate_speeding over limit',:]
df2 = new_filt_df.loc[new_filt_df['type'] != 'cite_rate_speeding over limit',:]
for col in ['asian/pacific islander', 'black', 'hispanic', 'white']:
  df2[col] = df2[col].map(lambda x: 1-x)

new_filt_df = pd.concat([df1,df2], ignore_index = True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
new_filt_df

,asian/pacific islander,black,hispanic,white,county,type
0,0.003871,0.001555,0.001768,0.002183,Hays County,cite_rate_speeding over limit
1,0.000000,0.000000,0.001085,0.000453,Wilson County,cite_rate_speeding over limit
2,0.000000,0.001004,0.001823,0.000850,Tarrant County,cite_rate_speeding over limit
3,0.000000,0.000598,0.001191,0.000570,Gregg County,cite_rate_speeding over limit
4,0.000000,0.000000,0.000923,0.001183,Jackson County,cite_rate_speeding over limit
...,...,...,...,...,...,...
808,1.000000,1.000000,0.957031,0.952077,Edwards County,cite_rate_speeding-10% or more above posted speed
809,1.000000,1.000000,0.982609,0.976703,Hansford County,cite_rate_speeding-10% or more above posted speed
810,1.000000,0.991979,0.998047,0.996962,Lynn County,cite_rate_speeding-10% or more above posted speed
811,1.000000,1.000000,1.000000,0.990566,Cochran County,cite_rate_speeding-10% or more above posted speed


In [ ]:
813*4

3252

In [ ]:
new_res_df = pd.DataFrame(columns = ['race','citation_rate', 'county','type'])
for col in ['asian/pacific islander', 'black', 'hispanic', 'white']:
  df = pd.DataFrame({'race':[col]*len(new_filt_df),'citation_rate':new_filt_df[col],'county':new_filt_df['county'],'type':new_filt_df['type']})
  new_res_df = pd.concat([new_res_df, df], ignore_index = True)
len(new_res_df)

3252

In [ ]:
new_res_df

,race,citation_rate,county,type
0,asian/pacific islander,0.003871,Hays County,cite_rate_speeding over limit
1,asian/pacific islander,0.000000,Wilson County,cite_rate_speeding over limit
2,asian/pacific islander,0.000000,Tarrant County,cite_rate_speeding over limit
3,asian/pacific islander,0.000000,Gregg County,cite_rate_speeding over limit
4,asian/pacific islander,0.000000,Jackson County,cite_rate_speeding over limit
...,...,...,...,...
3247,white,0.952077,Edwards County,cite_rate_speeding-10% or more above posted speed
3248,white,0.976703,Hansford County,cite_rate_speeding-10% or more above posted speed
3249,white,0.996962,Lynn County,cite_rate_speeding-10% or more above posted speed
3250,white,0.990566,Cochran County,cite_rate_speeding-10% or more above posted speed


In [ ]:
file_name = 'tableau_cite_rate_spd_cat.xlsx'
path = "/gdrive/MyDrive/traffic_stop/TX-county/summarystat/"
save_path = path + file_name
new_res_df.to_excel(save_path, index = False)

#### For other percentages

In [ ]:
file_name = 'spd_other_pct.csv'
path = "/gdrive/MyDrive/traffic_stop/TX-county/summarystat/"
pct_df = pd.read_csv((path+file_name))

In [ ]:
pct_df['type'].value_counts()

pct_spd_over_lmt       254
pct_srch               254
pct_contraband_srch    254
pct_stopped            254
Name: type, dtype: int64

In [ ]:
pct_df.head()

,asian/pacific islander,black,hispanic,white,county,type
0,0.476044,0.562650,0.582365,0.601788,Hays County,pct_spd_over_lmt
1,0.004008,0.006350,0.005467,0.005281,Hays County,pct_srch
2,0.333333,0.259259,0.127451,0.066421,Hays County,pct_contraband_srch
3,0.019996,0.056217,0.250528,0.673258,Hays County,pct_stopped
4,0.555944,0.638695,0.623600,0.656829,Wilson County,pct_spd_over_lmt


In [ ]:
len(pct_df)

1016

In [ ]:
(1016 - 86-13-5)*4

3648

In [ ]:
pct_df.isnull().sum()

asian/pacific islander    86
black                     13
hispanic                   4
white                      1
county                     0
type                       0
dtype: int64

In [ ]:
new_filt_df= pct_df[~pct_df.isnull().any(axis=1)]

In [ ]:
new_res_df = pd.DataFrame(columns = ['race','citation_rate','county','type'])
for col in ['asian/pacific islander', 'black', 'hispanic', 'white']:
  df = pd.DataFrame({'race':[col]*len(new_filt_df),'citation_rate':new_filt_df[col],'county':new_filt_df['county'],'type':new_filt_df['type']})
  new_res_df = pd.concat([new_res_df, df], ignore_index = True)
len(new_res_df)

3712

In [ ]:
new_res_df.head()

,race,citation_rate,county,type
0,asian/pacific islander,0.476044,Hays County,pct_spd_over_lmt
1,asian/pacific islander,0.004008,Hays County,pct_srch
2,asian/pacific islander,0.333333,Hays County,pct_contraband_srch
3,asian/pacific islander,0.019996,Hays County,pct_stopped
4,asian/pacific islander,0.555944,Wilson County,pct_spd_over_lmt


In [ ]:
file_name = 'tableau_other_pct.xlsx'
path = "/gdrive/MyDrive/traffic_stop/TX-county/summarystat/"
save_path = path + file_name
new_res_df.to_excel(save_path, index = False)

#### Total number of speeding stops and latitude & longitude

In [ ]:
# read in county_info
county_info = pd.read_csv('/gdrive/MyDrive/traffic_stop/table_county.csv')
  
# preprocessing county_info dataframe
# minus sign, delete celsius sign
county_info['Longitude'] = county_info['Longitude'].map(lambda x: '-' + x[1:])
for col in ['Latitude','Longitude']:
  county_info[col] = county_info[col].map(lambda x: x[:-1])
  county_info[col] = county_info[col].astype('float')

# we are focusing on Texas
county_info = county_info.loc[county_info['State'] == 'TX',:]
# rename county info's column
county_info = county_info[['State','County [2]','Latitude','Longitude','Population(2010)']]
county_info.rename(columns={"County [2]": "county"}, inplace = True)

county_info.head()

,State,county,Latitude,Longitude,Population(2010)
2523,TX,Anderson,31.841266,-95.661744,"58,458"
2524,TX,Andrews,32.312258,-102.640206,"14,786"
2525,TX,Angelina,31.251951,-94.611056,"86,771"
2526,TX,Aransas,28.104225,-96.977983,"23,158"
2527,TX,Archer,33.616305,-98.687267,"9,054"


In [ ]:
# get list of counties
filename = '/gdrive/MyDrive/traffic_stop/year_data_speeding_only/traffic_' + str(2016) + '.parquet'
df = pd.read_parquet(filename, engine = 'pyarrow')
county_lst = list(df['county_name'].unique())

In [ ]:
years = list(range(2006, 2018))
cnt_df = pd.Series([0]*254, index = county_lst)
for year in years:
  file_name = '/gdrive/MyDrive/traffic_stop/year_data_speeding_only/traffic_' + str(year) + '.parquet'
  df = pd.read_parquet(file_name, engine = 'pyarrow')
  for county in county_lst:
    cnt_df[county] += len(df.loc[df['county_name'] == county,:])
sum(cnt_df)

11842734

In [ ]:
cnt_df = pd.DataFrame(cnt_df).reset_index()
cnt_df.columns = ['county','speeding_stops']

In [ ]:
cnt_df

,county,speeding_stops
0,Hays County,81416
1,Wilson County,51924
2,Tarrant County,36822
3,Gregg County,63450
4,Jackson County,24486
...,...,...
249,Hansford County,9026
250,Lynn County,20477
251,Cochran County,1686
252,Loving County,171


In [ ]:
# make a 'new_county' column to correspond to the identifiers in the county_info csv
cnt_df['new_county'] = cnt_df['county'].map(lambda x: x.replace(' County', ''))
cnt_df['new_county'] = cnt_df['new_county'].replace('Dewitt','DeWitt')

In [ ]:
cnt_df

,county,speeding_stops,new_county
0,Hays County,81416,Hays
1,Wilson County,51924,Wilson
2,Tarrant County,36822,Tarrant
3,Gregg County,63450,Gregg
4,Jackson County,24486,Jackson
...,...,...,...
249,Hansford County,9026,Hansford
250,Lynn County,20477,Lynn
251,Cochran County,1686,Cochran
252,Loving County,171,Loving


In [ ]:
cnt_df_merged = cnt_df.merge(county_info, how = 'inner', left_on = 'new_county', right_on = 'county')
cnt_df_merged

,county_x,speeding_stops,new_county,State,county_y,Latitude,Longitude,Population(2010)
0,Hays County,81416,Hays,TX,Hays,30.061225,-98.029267,"157,107"
1,Wilson County,51924,Wilson,TX,Wilson,29.174303,-98.085899,"42,918"
2,Tarrant County,36822,Tarrant,TX,Tarrant,32.772040,-97.291291,"1,809,034"
3,Gregg County,63450,Gregg,TX,Gregg,32.486397,-94.816276,"121,730"
4,Jackson County,24486,Jackson,TX,Jackson,28.959802,-96.589080,"14,075"
...,...,...,...,...,...,...,...,...
249,Hansford County,9026,Hansford,TX,Hansford,36.272847,-101.356930,"5,613"
250,Lynn County,20477,Lynn,TX,Lynn,33.178412,-101.818493,"5,915"
251,Cochran County,1686,Cochran,TX,Cochran,33.608440,-102.830449,"3,127"
252,Loving County,171,Loving,TX,Loving,31.844936,-103.561229,82


In [ ]:
cnt_df_merged.drop(['county_x','State','county_y'], axis = 1, inplace = True)

In [ ]:
cnt_df_merged.rename(columns = {'new_county':'county'}, inplace = True)
cnt_df_merged

,speeding_stops,county,Latitude,Longitude,Population(2010)
0,81416,Hays,30.061225,-98.029267,"157,107"
1,51924,Wilson,29.174303,-98.085899,"42,918"
2,36822,Tarrant,32.772040,-97.291291,"1,809,034"
3,63450,Gregg,32.486397,-94.816276,"121,730"
4,24486,Jackson,28.959802,-96.589080,"14,075"
...,...,...,...,...,...
249,9026,Hansford,36.272847,-101.356930,"5,613"
250,20477,Lynn,33.178412,-101.818493,"5,915"
251,1686,Cochran,33.608440,-102.830449,"3,127"
252,171,Loving,31.844936,-103.561229,82


In [ ]:
type_df = pd.read_csv('/gdrive/MyDrive/traffic_stop/2014-2018.csv')
type_df = type_df[type_df['State']=='Texas']
type_df = type_df.filter(items=['Metropolitan Status', 'County Name'])
type_df
# transform column
df = cnt_df_merged.join(type_df.set_index('County Name'), on='county')

In [ ]:
sum(df['speeding_stops'])

11842734

In [ ]:
df

,speeding_stops,county,Latitude,Longitude,Population(2010),Metropolitan Status
0,81416,Hays,30.061225,-98.029267,"157,107",Metropolitan
1,51924,Wilson,29.174303,-98.085899,"42,918",Metropolitan
2,36822,Tarrant,32.772040,-97.291291,"1,809,034",Metropolitan
3,63450,Gregg,32.486397,-94.816276,"121,730",Metropolitan
4,24486,Jackson,28.959802,-96.589080,"14,075",Non core
...,...,...,...,...,...,...
249,9026,Hansford,36.272847,-101.356930,"5,613",Non core
250,20477,Lynn,33.178412,-101.818493,"5,915",Metropolitan
251,1686,Cochran,33.608440,-102.830449,"3,127",Non core
252,171,Loving,31.844936,-103.561229,82,Non core


In [ ]:
file_name = 'speeding_latlng.xlsx'
path = "/gdrive/MyDrive/traffic_stop/TX-county/summarystat/"
save_path = path + file_name
cnt_df_merged.to_excel(save_path, index = False)

### Population

In [ ]:
pop_df = pd.read_csv('/alldata.csv')
pop_df = pop_df.loc[pop_df['Age'] == 'All Ages',:]
keep_cols = ['County','FIPS', 'Age', 'Total','Total_Male', 'Total_Female','NH_White_Total','NH_Black_Total', 'NH_Asian_Total','NH_Other_Total','Hispanic_Total']
pop_df = pop_df[keep_cols]
pop_df.head()

,County,FIPS,Age,Total,Total_Male,Total_Female,NH_White_Total,NH_Black_Total,NH_Asian_Total,NH_Other_Total,Hispanic_Total
0,STATE OF TEXAS,0,All Ages,29001602,14405398,14596204,12034112,3439413,1429669,651510,11446898
97,ANDERSON COUNTY,1,All Ages,59025,36468,22557,34133,12495,315,1374,10708
194,ANDREWS COUNTY,3,All Ages,19279,9883,9396,6767,222,85,228,11977
291,ANGELINA COUNTY,5,All Ages,90989,44735,46254,54355,13650,919,1489,20576
388,ARANSAS COUNTY,7,All Ages,23710,11863,11847,15566,242,408,458,7036


In [ ]:
pop_df.columns

Index(['County', 'FIPS', 'Age', 'Total', 'Total_Male', 'Total_Female',
       'NH_White_Total', 'NH_Black_Total', 'NH_Asian_Total', 'NH_Other_Total',
       'Hispanic_Total'],
      dtype='object')

In [ ]:
def check_sum(row):
  if row['Total'] == row['NH_White_Total']+row['NH_Black_Total']+row['NH_Asian_Total']+row['NH_Other_Total']+row['Hispanic_Total']:
    return True
  else:
    return False
pop_df['check'] = pop_df.apply(lambda row: check_sum(row),axis = 1)
pop_df

,County,FIPS,Age,Total,Total_Male,Total_Female,NH_White_Total,NH_Black_Total,NH_Asian_Total,NH_Other_Total,Hispanic_Total,check
0,STATE OF TEXAS,0,All Ages,29001602,14405398,14596204,12034112,3439413,1429669,651510,11446898,True
97,ANDERSON COUNTY,1,All Ages,59025,36468,22557,34133,12495,315,1374,10708,True
194,ANDREWS COUNTY,3,All Ages,19279,9883,9396,6767,222,85,228,11977,True
291,ANGELINA COUNTY,5,All Ages,90989,44735,46254,54355,13650,919,1489,20576,True
388,ARANSAS COUNTY,7,All Ages,23710,11863,11847,15566,242,408,458,7036,True
...,...,...,...,...,...,...,...,...,...,...,...,...
24250,WOOD COUNTY,499,All Ages,45084,22573,22511,36874,2315,193,937,4765,True
24347,YOAKUM COUNTY,501,All Ages,8829,4426,4403,2625,62,24,102,6016,True
24444,YOUNG COUNTY,503,All Ages,19029,9381,9648,14495,236,65,362,3871,True
24541,ZAPATA COUNTY,505,All Ages,14196,6991,7205,758,11,29,27,13371,True


In [ ]:
pop_df['County'][1:].str.lower().map(lambda x: x[:-7].capitalize())

97       Anderson
194       Andrews
291      Angelina
388       Aransas
485        Archer
           ...   
24250        Wood
24347      Yoakum
24444       Young
24541      Zapata
24638      Zavala
Name: County, Length: 254, dtype: object

### ignore below

In [ ]:
# read in county_info
county_info = pd.read_csv('/gdrive/MyDrive/traffic_stop/table_county.csv')
  
# preprocessing county_info dataframe
# minus sign, delete celsius sign
county_info['Longitude'] = county_info['Longitude'].map(lambda x: '-' + x[1:])
for col in ['Latitude','Longitude']:
  county_info[col] = county_info[col].map(lambda x: x[:-1])
  county_info[col] = county_info[col].astype('float')

# we are focusing on Texas
county_info = county_info.loc[county_info['State'] == 'TX',:]
# rename county info's column
county_info = county_info[['State','County [2]','Latitude','Longitude','Population(2010)']]
county_info.rename(columns={"County [2]": "county"}, inplace = True)

county_info['county'] = county_info['county'].replace('DeWitt','Dewitt')
county_info
#county_info.head()

### Function for yearly traffic stop ratio decomposition
(Not finished)

In [ ]:
11498484+3218511+1876164+11269437

27862596

In [ ]:
pop_file = '/gdrive/MyDrive/traffic_stop/population_yearly/pop_' + str(2016) + '.csv'
pop_df = pd.read_csv(pop_file)
pop_df = pop_df.iloc[1:,:]
pop_df['County'] = pop_df['County'].str.lower().map(lambda x: x[:-7].capitalize())
if year == 2017:
  pop_df = pop_df.rename(columns = {'NH_White_Total':'white','NH_Black_Total':'black','NH_Asian_Total':'asian/pacific islander','Hispanic_Total':'hispanic'})
else:
  pop_df = pop_df.rename(columns = {'White_Total':'white','NH_Black_Total':'black','NH_Asian_Total':'asian/pacific islander','Hispanic_Total':'hispanic'})
pop_df

,County,FIPS,Age,Total,Total Male,Total Female,Anglo Total,Anglo Male,Anglo Female,Black Total,Black Male,Black Female,Other Total,Other Male,Other Female,Hispanic Total,Hispanic Male,Hispanic Female
1,State o,0,< 1 Year,403486,206037,197449,121028,62005,59023,46739,23728,23011,33246,17025,16221,202473,103279,99194
2,State o,0,1 Years,404970,207002,197968,122781,62925,59856,46492,23583,22909,33414,17146,16268,202283,103348,98935
3,State o,0,2 Years,402541,205902,196639,123317,63196,60121,45595,23161,22434,32796,16909,15887,200833,102636,98197
4,State o,0,3 Years,397056,203204,193852,122140,62607,59533,44363,22595,21768,32178,16670,15508,198375,101332,97043
5,State o,0,4 Years,391825,200363,191462,120608,61799,58809,43245,22033,21212,31628,16381,15247,196344,100150,96194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22180,Zavala,507,81 Years,41,16,25,3,2,1,0,0,0,0,0,0,38,14,24
22181,Zavala,507,82 Years,42,19,23,5,3,2,0,0,0,0,0,0,37,16,21
22182,Zavala,507,83 Years,31,12,19,5,2,3,0,0,0,0,0,0,26,10,16
22183,Zavala,507,84 Years,35,11,24,5,3,2,0,0,0,0,0,0,30,8,22


In [ ]:
years = list(range(2011, 2018))
res_df = pd.DataFrame()
for year in years:
  # speeding traffic stop file
  file_name = '/gdrive/MyDrive/traffic_stop/year_data_speeding_only/traffic_' + str(year) + '.parquet'
  df = pd.read_parquet(file_name, engine = 'pyarrow')
  df['county_name'] = df['county_name'].map(lambda x: x.replace(' County', ''))

  # population file
  pop_file = '/gdrive/MyDrive/traffic_stop/population_yearly/pop_' + str(year) + '.csv'
  pop_df = pd.read_csv(pop_file)
  pop_df = pop_df.iloc[1:,:]
  pop_df['County'] = pop_df['County'].str.lower().map(lambda x: x[:-7].capitalize())
  # note in population file, it is asian but not asian/pacific islander
  pop_df = pop_df.rename({'NH_White_Total':'white','NH_Black_Total':'black','NH_Asian_Total':'asian/pacific islander','Hispanic_Total':'hispanic'})

  for county in county_lst:
    df_county = df.loc[df['county_name'] == county,:]
    county_df = df_county['subject_race'].value_counts().reset_index()
    county_df = county_df.rename(columns = {'index': 'race','subject_race':'speeding_stops'})
    county_df['county'] = [county]*len(county_df)
    county_df['year'] = [int(year)]*len(county_df)
    county_df['population'] = county_df.apply(lambda row: pop_df.loc[pop_df['County'] == row['county'],row['race']], axis = 1)
    res_df = pd.concat([res_df, county_df], ignore_index = True)
res_df

KeyError: ignored

In [ ]:
pop_df['check'].value_counts().reset_index()

,index,check
0,True,255


#### Ignore: sanity checks

In [ ]:
type_df = pd.read_csv('/gdrive/MyDrive/traffic_stop/2014-2018.csv')
type_df

,geoid,County Name,State,metro code,Metropolitan Status,Total Population,"Total population, MOE",Disabiltiy Population,"Disability Population, MOE",Disabitiy Rate,"Disability Rate, MOE",Male Disabiltiy Rate,Female Disability Rate,Vision Disabiltiy Rate,Hearing Disabiltiy Rate,Cognative Disabitliy Rate,Mobility Disabiltiy Rate,Self Care Disabiltiy Rate,Independent Living,Disability Employment Rate,Disability Unemployment Rates,Out of the Labor Force Disability Rate,Veteran Disability Rate,Disability Poverty Rate,Disability Rate-Black/African America,Disability Rate-White,Disability Rate-Asian,Disability Rate-American Indian/Alaskan Native,Disability Rate-Native Hawaiian Pacific Islander,Disability Rate-Other race,Disability Rate-Two or more races,Disability Rate- Hispanic (of any race),Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35
0,0500000US01001,Autauga,Alabama,NaN,Metropolitan,54170,270,10201,791,18.8,1.4,19.2,18.5,2.8,5.1,8.2,10.5,4.0,8.5,34.3,4.1,61.6,32.1,23.6,24.7,17.3,5.3,41.4,54.8,11.1,30.2,16.1,NaN,NaN,NaN,NaN
1,0500000US01003,Baldwin,Alabama,NaN,Metropolitan,200929,427,27471,1447,13.7,0.7,14.3,13.1,2.0,4.4,4.6,7.8,2.1,5.6,28.8,4.8,66.4,31.4,16.7,14.7,13.4,15.0,41.8,NaN,1.4,15.5,4.5,NaN,NaN,NaN,NaN
2,0500000US01005,Barbour,Alabama,NaN,Micropolitan,23326,241,5144,368,22.1,1.6,20.4,23.6,5.2,5.5,9.8,13.8,5.1,10.6,25.7,5.3,69.0,45.9,33.6,22.3,23.2,4.2,0.0,NaN,11.6,12.7,12.4,NaN,NaN,NaN,NaN
3,0500000US01007,Bibb,Alabama,NaN,Metropolitan,20553,190,3260,377,15.9,1.8,15.2,16.5,2.4,2.8,5.2,10.8,4.1,8.7,21.0,6.6,72.4,37.4,18.9,18.1,15.3,NaN,6.1,NaN,NaN,22.0,3.3,NaN,NaN,NaN,NaN
4,0500000US01009,Blount,Alabama,NaN,Metropolitan,57207,155,8425,510,14.7,0.9,15.3,14.2,2.1,4.5,5.6,9.2,3.1,6.6,20.9,4.8,74.2,34.8,18.1,16.0,14.6,14.3,37.3,NaN,2.6,19.2,2.3,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,0500000US56037,Sweetwater,Wyoming,NaN,Micropolitan,44244,106,5107,453,11.5,1.0,11.2,11.9,2.5,4.3,4.5,5.7,1.6,4.7,47.6,7.8,44.7,25.3,16.9,0.8,12.0,11.2,5.4,0.0,3.4,9.4,8.5,NaN,NaN,NaN,NaN
3216,0500000US56039,Teton,Wyoming,NaN,Micropolitan,22865,44,1578,380,6.9,1.7,8.7,4.9,1.9,3.6,3.4,3.6,1.6,2.4,71.3,0.0,28.7,18.8,22.0,0.0,7.1,8.5,0.0,NaN,0.0,45.6,6.7,NaN,NaN,NaN,NaN
3217,0500000US56041,Uinta,Wyoming,NaN,Micropolitan,20575,56,3372,392,16.4,1.9,16.3,16.5,2.3,6.3,7.4,7.2,3.4,7.2,45.3,6.6,48.0,38.4,19.5,60.7,16.2,0.0,22.1,NaN,3.0,25.4,9.5,NaN,NaN,NaN,NaN
3218,0500000US56043,Washakie,Wyoming,NaN,Non core,8090,65,1378,215,17.0,2.6,17.2,16.9,3.8,6.3,6.3,7.7,2.3,4.7,48.2,10.8,41.0,27.8,14.7,0.0,17.4,0.0,61.5,NaN,13.1,10.0,19.9,NaN,NaN,NaN,NaN


In [ ]:
new_res_df.loc[new_res_df['county'] =='Austin County',:]

,race,citation_rate,county,type
231,asian/pacific islander,0.000000,Austin County,cite_rate_speeding over limit
420,asian/pacific islander,1.000000,Austin County,cite_rate_unsafe speed
560,asian/pacific islander,1.000000,Austin County,cite_rate_fail to control speed
791,asian/pacific islander,0.996606,Austin County,cite_rate_speeding-10% or more above posted speed
1044,black,0.000250,Austin County,cite_rate_speeding over limit
1233,black,1.000000,Austin County,cite_rate_unsafe speed
1373,black,1.000000,Austin County,cite_rate_fail to control speed
1604,black,0.997153,Austin County,cite_rate_speeding-10% or more above posted speed
1857,hispanic,0.000172,Austin County,cite_rate_speeding over limit
2046,hispanic,0.986842,Austin County,cite_rate_unsafe speed


In [ ]:
pd.DataFrame({'white':filtered_df['white'],'county':filtered_df['county'],'type':filtered_df['type']}).sort_values(by = 'county')

,white,county,type
45,0.009828,Anderson County,cite_rate_unsafe speed
553,0.002929,Anderson County,cite_rate_speeding-10% or more above posted speed
807,0.000488,Anderson County,cite_rate_speeding over limit
299,0.024390,Anderson County,cite_rate_fail to control speed
970,0.005279,Andrews County,cite_rate_speeding over limit
...,...,...,...
615,0.088670,Zapata County,cite_rate_speeding-10% or more above posted speed
869,0.001350,Zapata County,cite_rate_speeding over limit
591,0.079511,Zavala County,cite_rate_speeding-10% or more above posted speed
845,0.000913,Zavala County,cite_rate_speeding over limit


In [ ]:
for year in list(range(2006, 2018)):
  file_name = '/gdrive/MyDrive/traffic_stop/year_data_speeding_only/traffic_' + str(year) + '.parquet'
  df = pd.read_parquet(file_name, engine = 'pyarrow')
  print(year)
  df = df.loc[(df['county_name'] == 'Montgomery County') & (df['violation'] == 'unsafe speed') & (df['speeding_only'].isin(['speeding-repeated_entries','speeding-1']) &(df['subject_race'] == 'asian/pacific islander')),:]
  print(df[['subject_race','citation_issued']])

2006
Empty DataFrame
Columns: [subject_race, citation_issued]
Index: []
2007
                  subject_race  citation_issued
288819  asian/pacific islander                1
291586  asian/pacific islander                1
293531  asian/pacific islander                1
298316  asian/pacific islander                1
2008
                  subject_race  citation_issued
469325  asian/pacific islander                1
2009
                  subject_race  citation_issued
45194   asian/pacific islander                1
114967  asian/pacific islander                1
356803  asian/pacific islander                1
719187  asian/pacific islander                1
747979  asian/pacific islander                1
2010
                  subject_race  citation_issued
256062  asian/pacific islander                1
256571  asian/pacific islander                1
590491  asian/pacific islander                1
955234  asian/pacific islander                1
2011
                  subject_race  citatio

In [ ]:
filtered_df.head()

,asian/pacific islander,black,hispanic,white,county,type
0,0.0,0.000000,0.073529,0.017167,Hays County,cite_rate_unsafe speed
1,0.0,0.000000,0.008475,0.010563,Wilson County,cite_rate_unsafe speed
3,0.0,0.011236,0.000000,0.023750,Gregg County,cite_rate_unsafe speed
4,0.0,0.000000,0.052632,0.051724,Jackson County,cite_rate_unsafe speed
5,0.0,0.014493,0.000000,0.011287,Jasper County,cite_rate_unsafe speed
